# Results

In [ ]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd

In [6]:
def percentage_improvement(old_error, new_error):
    """
    Calculates the percentage improvement between two error values.

    Args:
      old_error: The old error value.
      new_error: The new error value.

    Returns:
      The percentage improvement, as a float.
    """

    # return (new_error - old_error) / old_error * 100
    return (old_error - new_error) / old_error * 100


if __name__ == "__main__":
    # data = [(2.040, 0.608), (40.797, 0.040), (0.916, 0.061), (58.580, 0.118)]
    # data = [(2.194, 0.556), (2.613, 0.787), (260.249, 0.112), (106.582, 0.080)]

    # Complex medium 1000Hz (real, Imag), 1500Hz (real, Imag)
    data = [(3.190, 0.087), (2.632, 0.045), (1.322, 0.043), (3.486, 0.094)]

    for i in data:
        result = percentage_improvement(old_error=i[0], new_error=i[1])

        print("The percentage improvement is", result, "%.")


# Complex NN

In [4]:
import os
import pandas as pd

from numpy import pi, power, savetxt

from pinn.complex.ComplexNN import PINN_CMPLX
from utils.dataset_utils import plot_output_complex

def complex_medium_delany_bazley(f: int = 100, c: int = 340, rho: float = 1.225, e: int = 250) -> complex:
    """
    SPEED OF SOUND IN COMPLEX MEDIUM DELANY BAZLEY
    :param f: Frequency (Hz)
    :param c: Speed of sound (m/s)
    :param rho: Constant
    :param e: Flow resistance (rayls/m)
    :return cw: Speed of sound in Complex domain
    """
    omega = 2 * pi * f
    k0 = omega / c
    X = rho * f / e
    kw = k0 * (1 + (0.0978 * power(X, -0.7)) - (1j * 0.189 * power(X, -0.595)))
    cw = omega/kw
    return cw

def export_data(arr, export_path, filename):
    if not os.path.exists(export_path):
        os.makedirs(export_path)

    print(len(arr))
    savetxt(f'{export_path}/{filename}.csv', arr, delimiter=',')

    # Read the CSV file
    df = pd.read_csv(f'{export_path}/{filename}.csv', header=None)
    df= df.T
    df.columns = ['test_data', 'gt_real', 'nn_out_real', 'gt_imag', 'nn_out_imag']


    # Save the transposed DataFrame to a CSV file
    df.to_csv(f'{export_path}/{filename}.csv', index=False)
    print(f"Exported data to: {export_path}/{filename}.csv")
    # df.to_csv(f'{export_path}/data_relative_error.csv')
    # print(f"Exported data to: {export_path}/data_relative_error.csv")


if __name__ == '__main__':
    DLRS = True

    # Variables associated with physics
    F = 1500                                 # Frequency
    C = 340                                 # Speed of sound
    RHO = 1.225
    E = 250
    CW = complex_medium_delany_bazley(F, C, RHO, E)

    X_LB = 0.0                                # Left boundary
    X_RB = 1.0                                # Right boundary
    P_LB = 1.0 + 0.0j                         # Pressure at the left boundary
    P_RB = -1.0 + 0.0j                        # Pressure at the right boundary
    K = 2 * pi * F / CW                    # Wave number

    if DLRS:
        experiment_name = f'dlrs/{F}Hz'
        filename = f'dlrs_{F}Hz'
    else:
        experiment_name = f'std/{F}Hz'
        filename = f'std_{F}Hz'

    # Initialize the PINN model
    NN = PINN_CMPLX(frequency=F, exp_name=experiment_name, cw=CW, x_lb=X_LB, x_rb=X_RB, p_lb=P_LB, p_rb=P_RB)
    NN.load_model(path=f'./weights/CMPLX/complex_medium/{experiment_name}_last')

    export_dict = plot_output_complex(F, K, P_LB, P_RB, NN)

    # test_data = export_dict['test_data']
    # S_r = export_dict['ground_truth_real']
    # D_r = export_dict['nn_output_real']
    # S_i = export_dict['ground_truth_imaginary']
    # D_i = export_dict['nn_output_imaginary']

    # arr = [test_data, S_r, D_r, S_i, D_i]
    # export_path =  f'outputs/exports/complex/complex_medium/{experiment_name}'
    # export_path = r"C:\Users\Ashwi\Documents\Research\POMA\graphs\delaney_bazley\relative_error/"

    # export_data(arr, export_path, filename)



# Tensorboard logs

In [1]:
def export(filepath, outfile_path, filename, tag='loss'):
    # Retrieve the events data
    data = []
    for event in tf.compat.v1.train.summary_iterator(filepath):
        for value in event.summary.value:
            # event.wall_time, event.step, value.tag, tf.make_ndarray(value.tensor)
            if value.tag == tag:
                data.append([event.step, tf.make_ndarray(value.tensor)])
                # data.append([event.step, value.simple_value])  # pytorch exports

    df = pd.DataFrame(data, columns=['epoch', f'{tag.lower()}'])

    # Save DataFrame to a CSV file
    output_file = f"{outfile_path}/{filename}.csv"
    df.to_csv(output_file, index=False)
    print(f"Tensorboard logs successfully extracted and saved to {output_file}.")


def gather_file_paths(directory):
    path_list = []
    # parse directories
    for root, dirs, files in os.walk(directory):
        if len(files) > 0:
            for file in files:
                # print("Current directory:", root)
                # print("Subdirectories:", dirs)
                # print("File:", file)
                # print("--------------------")
                path_list.append(os.path.join(root, file))
    return path_list


def get_tags(filename):
    tags = []
    for event in tf.compat.v1.train.summary_iterator(filename):
        for value in event.summary.value:
            # event.wall_time, event.step, value.tag, tf.make_ndarray(value.tensor)
            if value.tag in tags:
                break
            else:
                tags.append(value.tag)

    return tags

In [29]:
log_dir = r"logs/complex/complex_medium/std/1500Hz"

filepaths = gather_file_paths(log_dir)

for path in filepaths:
    tags = get_tags(path)

    if len(tags) != 0:
        print(tags)
        break


In [30]:
# Export for PINNs dataset
for path in filepaths:
    parent_dir = os.path.dirname(path[5:])[:-6]
    export(path, outfilename=parent_dir, tag='loss')

# Express export

In [5]:
f = 1500
DLRS = True

if DLRS:
    filename = f'dlrs_{f}Hz'
    log_dir = fr"logs/complex/complex_medium/dlrs/{f}Hz"
else:
    filename = f'std_{f}Hz'
    log_dir = fr"logs/complex/complex_medium/std/{f}Hz"

outfile_path = r"C:\Users\Ashwi\Documents\Research\POMA\graphs\delaney_bazley\loss/"

filepaths = gather_file_paths(log_dir)

for path in filepaths:
    export(path, outfile_path, filename,tag='loss')
